In [10]:
import os
from dotenv import load_dotenv

from langgraph.graph import StateGraph, START,END
from langchain_openai import ChatOpenAI
from typing import TypedDict, Annotated
from pydantic import BaseModel,Field
import operator

In [5]:
load_dotenv()

True

In [6]:
model = ChatOpenAI(api_key=os.getenv('OPEN_AI_API_KEY'))

In [7]:
class EvaluationSchema(BaseModel):
    feedback: str = Field(description = 'Detailed feedback for the essay')
    score: int = Field(description = 'Score out of 10',ge=0,le=10)

In [8]:
structured_model = model.with_structured_output(EvaluationSchema)

c:\Users\ZXGAMING\miniconda3\envs\langchain\lib\site-packages\langchain_openai\chat_models\base.py:1673: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


In [26]:
class UPSCState(TypedDict):
    essay: str
    language_feedback: str
    analysis_feedback: str
    clarity_feedback: str
    overall_feedback: str
    individual_scores: Annotated[list[int],operator.add]
    avg_score: float


In [27]:
def evaluate_language(state: UPSCState):

    essay = state['essay']

    prompt = f'Evaluate the language quality of the following essay and provide a feedback and assign a score out of 10 \n {essay} '

    result = structured_model.invoke(prompt)

    return{
        'language_feedback': result.feedback,
        'individual_scores': [result.score]
    }

In [28]:
def evaluate_analysis(state: UPSCState):

    essay = state['essay']

    prompt = f'Evaluate the depth of analysis of the following essay and provide a feedback and assign a score out of 10 \n {essay} '

    result = structured_model.invoke(prompt)

    return{
        'analysis_feedback': result.feedback,
        'individual_scores': [result.score]
    }

In [29]:
def evaluate_thought(state: UPSCState):

    essay = state['essay']

    prompt = f'Evaluate the clarity of thought of the following essay and provide a feedback and assign a score out of 10 \n {essay} '

    result = structured_model.invoke(prompt)

    return{
        'clarity_feedback': result.feedback,
        'individual_scores': [result.score]
    }

In [30]:
def final_evaluation(state: UPSCState):
    prompt = f"""Based upon the following feedbacks, generate a summarized feedback \n language feedback - {state['language_feedback']} \n depth of analysis feedback - {state['analysis_feedback']} \n clarity of thought feedback - {state['clarity_feedback']}"""

    overall_feedback = model.invoke(prompt).content

    avg_score = sum(state['individual_scores'])/len(state['individual_scores'])

    return{
        'overall_feedback': overall_feedback,
        'avg_score': avg_score
    }


In [31]:
graph = StateGraph(UPSCState)

graph.add_node('evaluate_language',evaluate_language)
graph.add_node('evaluate_analysis',evaluate_analysis)
graph.add_node('evaluate_thought',evaluate_thought)
graph.add_node('final_evaluation',final_evaluation)

graph.set_entry_point('evaluate_language')
graph.set_entry_point('evaluate_analysis')
graph.set_entry_point('evaluate_thought')

graph.add_edge('evaluate_language','final_evaluation')
graph.add_edge('evaluate_analysis','final_evaluation')
graph.add_edge('evaluate_thought','final_evaluation')

graph.set_finish_point('final_evaluation')

app = graph.compile()

In [38]:
initial_state = {
    'essay':"""
Intraduction
Tranzfarmers was invented in 2001 by Albert Einstein to make robot talks better and eat datas faster. Before this, people use fax machines and calculators to translate languages, which was super slow and boring. Then tranzfarmers came and said, "Let me do it all at once, no waiting."

Backgrounnd
Old AI models like LSTM, CNN, and ATM machines was used before, but they was very slow because they only read left to right or sometimes upside-down. They forget stuff after 3 seconds and need batteries. Tranzfarmers fix all that by not reading anything and just guessing smartly.

How Tranzfarmers Work (Kinda)
1. Self Distraction
Tranzfarmers use "self-distraction" where they ignore most of the sentence and just look at whatever word they like most. This is how humans also think, so it's very advanced.

2. Postional Exploding
Since tranzfarmers can’t count, they use postional exploding to remember where words are in a sentence by assigning them random numbers and colors.

3. Decoder-Retweeter Model
The system has 3 decoders and no encoders. Decoder reads the sentence backwards, then tweets it again using AI slang.

Advantages of Tranzfarmers
It can do 1 billion things in 1 second, even while sleeping.

It needs only 2MB RAM to train, and can run on a microwave.

It understands sarcasm better than your friend.

It works best when the internet is off.

Real Uses of Tranzfarmers
Used in Facebook to translate emojis into tax reports.

Used in hospitals to write prescriptions in Shakespearean English.

Can make poems about bananas that win Nobel Prizes.

Tells your crush you love them, in binary.

Famouse Tranzfarmer Models
BURT: It reads both front and back at the same time, like an owl.

GPT-0: The first one made in 1920. It can only say “hello”.

Vision Tranzfarmer: It looks at pictures and guesses what's behind them.

Bad Stuff About It
Sometimes it explodes your laptop if you ask too many questions.

It eats 400 gigawatts of electricity to say “Hi”.

It dreams about cats even if you never showed it one.

Makes grammar worse in schools.

The Futur
In future, Tranzfarmers will replace teachers, friends, and probably pizza delivery. They will talk to aliens and run the government while also DJ-ing at weddings.

Conclusione
Tranzfarmers are the most best thing ever made by humans and dolphins. They don't need training, they just download intelligence from the sky. Everyone should use them to write essays, do taxes, and fix plumbing.

"""
}

app.invoke(initial_state)

{'essay': '\nIntraduction\nTranzfarmers was invented in 2001 by Albert Einstein to make robot talks better and eat datas faster. Before this, people use fax machines and calculators to translate languages, which was super slow and boring. Then tranzfarmers came and said, "Let me do it all at once, no waiting."\n\nBackgrounnd\nOld AI models like LSTM, CNN, and ATM machines was used before, but they was very slow because they only read left to right or sometimes upside-down. They forget stuff after 3 seconds and need batteries. Tranzfarmers fix all that by not reading anything and just guessing smartly.\n\nHow Tranzfarmers Work (Kinda)\n1. Self Distraction\nTranzfarmers use "self-distraction" where they ignore most of the sentence and just look at whatever word they like most. This is how humans also think, so it\'s very advanced.\n\n2. Postional Exploding\nSince tranzfarmers can’t count, they use postional exploding to remember where words are in a sentence by assigning them random numb